## Selección de Ejecución de Inversiones Pantaleón

Pantaleón está considerando seis posibles proyectos de construcción durante los próximos 4 años. Fox puede emprender cualquiera de los proyectos en parte o en su totalidad. La ejecución parcial de un proyecto prorrateará proporcionalmente tanto el rendimiento como los desembolsos de efectivo. Los rendimientos (valor presente) y los desembolsos de efectivo para los proyectos se dan en la siguiente tabla.


| Proyecto | Desembolso Año 1 ($1000) | Desembolso Año 2  ($1000) | Desembolso Año 3  ($1000) | Desembolso Año 4 ($1000)| Rendimiento ($1000) |
|:--:|:--:|:--:|:--:|:--:|:--:|
| 1 | 10.5 | 14.4 | 2.2 | 2.4 | 32.40 |
| 2 | 8.3  | 12.6 | 9.5 | 3.1 | 35.80 |
| 3 | 10.2 | 14.2 | 5.6 | 4.2 | 17.75 |
| 4 | 7.2  | 10.5 | 7.5 | 5.0 | 14.80 |
| 5 | 12.3 | 10.1 | 8.3 | 6.3 | 18.20 |
| 6 | 9.2  | 7.8  | 6.9 | 5.1 | 12.35 |
| Fondos Disponibles ($1000) | 60.0 | 70.0 | 35.0 | 20.0 |  |

Formule el problema como un programa lineal, y determine la combinación óptima de proyectos que maximice el rendimiento total utilizando Gurobi.
Pase por alto el valor en el tiempo del dinero.

### Formulación Matemática

#### Conjuntos y notación
- Sea $p$ el número de inversiones o proyectos que se consideran.
- Sea $P = \{1,2,...,p\}$ el conjunto de inversiones o proyectos.
- Sea $a$ el número de años que se consideran.
- Sea $A = \{1,2,...,a\}$ el conjunto de años en el análisis.
- Sea $f_j$ los fondos disponibles para invertir en el año $j$, $\forall j \in A$
- Sea $d_{i,j}$ el desembolso que requiere el projecto $i$ en el año $j$, $\forall i \in P, \forall j \in A$
- Sea $r_i$ el rendimiento (valor presente) ofrecido por el projecto $i$, $\forall i \in P$

#### Variables de decisión
- Sea $x_i$ la ejecución parcial o total del proyecto $i$, $\forall i \in P$

#### Función Objetivo
La función objetivo se plantea para maximizar el rendimiento total del portafolio de proyectos a seleccionar.

$\begin{align}
\max\limits_{\forall i \in P} \sum_{i=1}^{p} r_i x_i
\end{align}$

Sujeta a:

$\begin{align}
\sum_{i=1}^{p} d_{i,j} x_i \leq f_j, \forall i \in P, \forall j \in A \\
x_i \in [0,1], \forall i \in P
\end{align}$

In [1]:
# librerías
import pandas as pd
import numpy as np
from gurobipy import *

In [2]:
# datos de proyectos
proyectos, rendimientos = multidict({
    'proyecto_1': 32.40,
    'proyecto_2': 35.80,
    'proyecto_3': 17.75,
    'proyecto_4': 14.80,
    'proyecto_5': 18.20,
    'proyecto_6': 12.35
})

# datos de disponibilidad de fondos
años, fondos = multidict({
    '1': 60.0,
    '2': 70.0,
    '3': 35.0,
    '4': 20.0,
})

# datos de flujos
flujos = {
    ('proyecto_1','1'): 10.5,
    ('proyecto_1','2'): 14.4,
    ('proyecto_1','3'): 2.2,
    ('proyecto_1','4'): 2.4,
    ('proyecto_2','1'): 8.3,
    ('proyecto_2','2'): 12.6,
    ('proyecto_2','3'): 9.5,
    ('proyecto_2','4'): 3.1,
    ('proyecto_3','1'): 10.2,
    ('proyecto_3','2'): 14.2,
    ('proyecto_3','3'): 5.6,
    ('proyecto_3','4'): 4.2,
    ('proyecto_4','1'): 7.2,
    ('proyecto_4','2'): 10.5,
    ('proyecto_4','3'): 7.5,
    ('proyecto_4','4'): 5.0,
    ('proyecto_5','1'): 12.3,
    ('proyecto_5','2'): 10.1,
    ('proyecto_5','3'): 8.3,
    ('proyecto_5','4'): 6.3,
    ('proyecto_6','1'): 9.2,
    ('proyecto_6','2'): 7.8,
    ('proyecto_6','3'): 6.9,
    ('proyecto_6','4'): 5.1
}

In [3]:
# inicialización de modelo 
f = Model('Inversiones')

# creación de variables de decisión: ejecución por proyecto
ejecucion = f.addVars(proyectos, name="ejecutar")

# creación de restricciones
res = f.addConstrs(((sum(flujos[p,a]*ejecucion[p] for p in proyectos) <= fondos[a]) for a in años), name='R')
res1 = f.addConstrs((ejecucion[p] <= 1 for p in proyectos))

Set parameter Username


In [4]:
# el objetivo es maximizar el rendimiento total
f.setObjective(ejecucion.prod(rendimientos),GRB.MAXIMIZE)

In [5]:
# guardar modelo para inspección
f.write('inversiones.lp')
f.display()

Maximize
32.4 ejecutar[proyecto_1] + 35.8 ejecutar[proyecto_2] + 17.75 ejecutar[proyecto_3]
+ 14.8 ejecutar[proyecto_4] + 18.2 ejecutar[proyecto_5] + 12.35 ejecutar[proyecto_6]
Subject To
R[1]: 10.5 ejecutar[proyecto_1] + 8.3 ejecutar[proyecto_2] + 10.2 ejecutar[proyecto_3]
 + 7.2 ejecutar[proyecto_4] + 12.3 ejecutar[proyecto_5] + 9.2 ejecutar[proyecto_6] <= 60
R[2]: 14.4 ejecutar[proyecto_1] + 12.6 ejecutar[proyecto_2] + 14.2 ejecutar[proyecto_3]
 + 10.5 ejecutar[proyecto_4] + 10.1 ejecutar[proyecto_5] + 7.8 ejecutar[proyecto_6] <= 70
R[3]: 2.2 ejecutar[proyecto_1] + 9.5 ejecutar[proyecto_2] + 5.6 ejecutar[proyecto_3] +
 7.5 ejecutar[proyecto_4] + 8.3 ejecutar[proyecto_5] + 6.9 ejecutar[proyecto_6] <= 35
R[4]: 2.4 ejecutar[proyecto_1] + 3.1 ejecutar[proyecto_2] + 4.2 ejecutar[proyecto_3] +
 5.0 ejecutar[proyecto_4] + 6.3 ejecutar[proyecto_5] + 5.1 ejecutar[proyecto_6] <= 20
  R4: ejecutar[proyecto_1] <= 1
  R5: ejecutar[proyecto_2] <= 1
  R6: ejecutar[proyecto_3] <= 1
  R7: ejecutar[p

In [6]:
# motor de optimización
f.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x9bac47f8
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 2 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1545455e+02   9.727273e+00   0.000000e+00      0s
       3    1.1606111e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.160611111e+02


In [7]:
# despliegue de valor óptimo de las variables de decisión
for v in f.getVars():
    if(abs(v.x)>1e-6):
        print(v.varName, round(v.x,2))

# valor óptimo de rendimiento total
print('rendimiento total', round(f.objVal,2))

ejecutar[proyecto_1] 1.0
ejecutar[proyecto_2] 1.0
ejecutar[proyecto_3] 1.0
ejecutar[proyecto_4] 1.0
ejecutar[proyecto_5] 0.84
rendimiento total 116.06


Suponga que si se emprende una parte del proyecto 2, entonces debe emprenderse por lo menos una parte igual del proyecto 6. Modifique la formulación del modelo y determine la nueva solución óptima.

In [8]:
# inicialización de modelo 
f = Model('Inversiones')

# creación de variables de decisión: ejecución por proyecto
ejecucion = f.addVars(proyectos, name="ejecutar")

# creación de restricciones
res = f.addConstrs(((sum(flujos[p,a]*ejecucion[p] for p in proyectos) <= fondos[a]) for a in años), name='R')
res1 = f.addConstrs((ejecucion[p] <= 1 for p in proyectos))
res2 = f.addConstr(ejecucion['proyecto_6'] >= ejecucion['proyecto_2'],name='res_p6')

In [9]:
# el objetivo es maximizar el rendimiento total
f.setObjective(ejecucion.prod(rendimientos),GRB.MAXIMIZE)

# guardar modelo para inspección
f.write('inversiones_1.lp')
f.display()

Maximize
32.4 ejecutar[proyecto_1] + 35.8 ejecutar[proyecto_2] + 17.75 ejecutar[proyecto_3]
+ 14.8 ejecutar[proyecto_4] + 18.2 ejecutar[proyecto_5] + 12.35 ejecutar[proyecto_6]
Subject To
R[1]: 10.5 ejecutar[proyecto_1] + 8.3 ejecutar[proyecto_2] + 10.2 ejecutar[proyecto_3]
 + 7.2 ejecutar[proyecto_4] + 12.3 ejecutar[proyecto_5] + 9.2 ejecutar[proyecto_6] <= 60
R[2]: 14.4 ejecutar[proyecto_1] + 12.6 ejecutar[proyecto_2] + 14.2 ejecutar[proyecto_3]
 + 10.5 ejecutar[proyecto_4] + 10.1 ejecutar[proyecto_5] + 7.8 ejecutar[proyecto_6] <= 70
R[3]: 2.2 ejecutar[proyecto_1] + 9.5 ejecutar[proyecto_2] + 5.6 ejecutar[proyecto_3] +
 7.5 ejecutar[proyecto_4] + 8.3 ejecutar[proyecto_5] + 6.9 ejecutar[proyecto_6] <= 35
R[4]: 2.4 ejecutar[proyecto_1] + 3.1 ejecutar[proyecto_2] + 4.2 ejecutar[proyecto_3] +
 5.0 ejecutar[proyecto_4] + 6.3 ejecutar[proyecto_5] + 5.1 ejecutar[proyecto_6] <= 20
  R4: ejecutar[proyecto_1] <= 1
  R5: ejecutar[proyecto_2] <= 1
  R6: ejecutar[proyecto_3] <= 1
  R7: ejecutar[p

In [10]:
# motor de optimización
f.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 11 rows, 6 columns and 32 nonzeros
Model fingerprint: 0xe4922a35
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+01]
Presolve removed 8 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 14 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1545455e+02   9.727273e+00   0.000000e+00      0s
       4    1.1367778e+02   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.136777778e+02


In [11]:
# despliegue de valor óptimo de las variables de decisión
for v in f.getVars():
    if(abs(v.x)>1e-6):
        print(v.varName, round(v.x,2))

# valor óptimo de rendimiento total
print('rendimiento total', round(f.objVal,2))

ejecutar[proyecto_1] 1.0
ejecutar[proyecto_2] 1.0
ejecutar[proyecto_3] 1.0
ejecutar[proyecto_4] 1.0
ejecutar[proyecto_5] 0.03
ejecutar[proyecto_6] 1.0
rendimiento total 113.68
